# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [4]:
df = pd.DataFrame(data.groupby(['CustomerID', 'ProductName']).Quantity.sum())
df = df.reset_index()

In [5]:
df

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1
...,...,...,...
63623,98200,Vol Au Vents,50
63624,98200,Wasabi Powder,25
63625,98200,Wine - Fume Blanc Fetzer,25
63626,98200,Wine - Hardys Bankside Shiraz,25


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [6]:
product_by_customer = df.pivot_table(index=["CustomerID"], columns=["ProductName"], values=["Quantity"]).fillna(0)
#instead of doing transpose, already assign productName to columns
product_by_customer

Quantity                                    \
ProductName Anchovy Paste - 56 G Tube Appetizer - Mini Egg Roll, Shrimp   
CustomerID                                                                
33                                0.0                               0.0   
200                               0.0                               0.0   
264                               0.0                               0.0   
356                               0.0                               0.0   
412                               0.0                               0.0   
...                               ...                               ...   
97928                             0.0                               0.0   
98069                             0.0                               0.0   
98159                             0.0                               0.0   
98185                             0.0                               0.0   
98200                             0.0                               0.0   

                                                                 \
ProductName Appetizer - Mushroom Tart Appetizer - Sausage Rolls   
CustomerID                                                        
33                                0.0                       0.0   
200                               0.0                       0.0   
264                               0.0                       0.0   
356                               0.0                       0.0   
412                               0.0                       0.0   
...                               ...                       ...   
97928                             0.0                      25.0   
98069                             0.0                      25.0   
98159                             0.0                       0.0   
98185                            25.0                      25.0   
98200                             0.0                       0.0   

                                                               \
ProductName Apricots - Dried Apricots - Halves Apricots Fresh   
CustomerID                                                      
33                       1.0               0.0            0.0   
200                      0.0               0.0            0.0   
264                      0.0               1.0            1.0   
356                      0.0               0.0            0.0   
412                      1.0               0.0            0.0   
...                      ...               ...            ...   
97928                    0.0              50.0            0.0   
98069                    0.0              25.0            0.0   
98159                    0.0               0.0            0.0   
98185                    0.0              25.0            0.0   
98200                    0.0              25.0           25.0   

                                                                          ...  \
ProductName Arizona - Green Tea Artichokes - Jerusalem Assorted Desserts  ...   
CustomerID                                                                ...   
33                          0.0                    0.0               1.0  ...   
200                         0.0                    0.0               0.0  ...   
264                         0.0                    0.0               0.0  ...   
356                         0.0                    0.0               0.0  ...   
412                         0.0                    0.0               0.0  ...   
...                         ...                    ...               ...  ...   
97928                      25.0                    0.0               0.0  ...   
98069                       0.0                    0.0              25.0  ...   
98159                       0.0                    0.0               0.0  ...   
98185                       0.0                    0.0               0.0  ...   
98200                       0.0                    0.0              25.0  .

In [7]:
#normalizing data

product_by_customer = product_by_customer.div(product_by_customer.sum(axis=1), axis=0)

## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [8]:
def customer_sim(summary, metric='euclidean'):
    pairwise = pd.DataFrame(
        1/(1 + squareform(pdist(summary, metric))),
        columns = summary.index,
        index = summary.index
    )
    return pairwise

In [9]:
euclidean_sim = customer_sim(product_by_customer)
euclidean_sim

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.854082,0.859145,0.852783,0.856118,0.853239,0.849549,0.848801,0.851224,0.849892,...,0.855130,0.846812,0.850301,0.841254,0.852221,0.846485,0.852848,0.847637,0.856465,0.851614
200,0.854082,1.000000,0.857935,0.855900,0.856445,0.853129,0.863562,0.850521,0.857381,0.856621,...,0.864397,0.856014,0.862447,0.854629,0.856670,0.854046,0.868990,0.857312,0.861508,0.857184
264,0.859145,0.857935,1.000000,0.851801,0.860050,0.851262,0.855847,0.846677,0.854206,0.848997,...,0.861268,0.850766,0.854178,0.848507,0.858909,0.860153,0.859773,0.854246,0.864699,0.853668
356,0.852783,0.855900,0.851801,1.000000,0.852571,0.849431,0.853831,0.851902,0.847607,0.846328,...,0.857171,0.848015,0.852222,0.847706,0.851413,0.851289,0.851219,0.852260,0.860721,0.854377
412,0.856118,0.856445,0.860050,0.852571,1.000000,0.855755,0.858976,0.857221,0.854857,0.855991,...,0.859117,0.862847,0.854943,0.850044,0.858385,0.856629,0.855052,0.856608,0.865269,0.862253
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.846485,0.854046,0.860153,0.851289,0.856629,0.855545,0.856079,0.851250,0.853632,0.849471,...,0.857702,0.854124,0.850987,0.851150,0.862796,1.000000,0.862919,0.855458,0.858954,0.851527
98069,0.852848,0.868990,0.859773,0.851219,0.855052,0.852574,0.857118,0.851869,0.855343,0.855701,...,0.859570,0.855797,0.855409,0.847469,0.861720,0.862919,1.000000,0.859010,0.864271,0.858525
98159,0.847637,0.857312,0.854246,0.852260,0.856608,0.852782,0.854378,0.851396,0.855557,0.851331,...,0.859624,0.858006,0.853659,0.845236,0.855405,0.855458,0.859010,1.000000,0.861266,0.854102


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [10]:
similar_customers = list(euclidean_sim[33].sort_values(ascending=False)[1:].head().index)
similar_customers
#returning the list that containt customer IDs becuase matrix is done from IDs

[60862, 27672, 6001, 79458, 33759]

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [11]:
similar_customer_records = df[df.CustomerID.isin(similar_customers)]
similar_customer_records
#from df taking only customers with these IDs

,CustomerID,ProductName,Quantity
4418,6001,Apricots - Halves,2
4419,6001,Apricots Fresh,2
4420,6001,Arizona - Green Tea,2
4421,6001,Assorted Desserts,2
4422,6001,"Beef - Chuck, Boneless",2
...,...,...,...
50794,79458,"Wine - Cahors Ac 2000, Clos",21
50795,79458,Wine - Crozes Hermitage E.,21
50796,79458,Wine - Pinot Noir Latour,21
50797,79458,Wine - Redchard Merritt,42


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [12]:
grouped = pd.DataFrame(similar_customer_records.groupby('ProductName').Quantity.sum())
product_records = grouped.sort_values('Quantity', ascending=False).reset_index()
product_records

,ProductName,Quantity
0,Wine - Redchard Merritt,59
1,Cassis,58
2,Bread - Calabrese Baguette,45
3,Wine - Crozes Hermitage E.,45
4,"Thyme - Lemon, Fresh",42
...,...,...
283,Flour - Teff,2
284,Pop Shoppe Cream Soda,2
285,Pomello,2
286,Water - Spring Water 500ml,2


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [13]:
merged = pd.merge(product_records, pd.DataFrame(product_by_customer.T[33]), on='ProductName')
merged.columns = ['ProductName', 'Quantity', 'Purchased']
customer_orders = merged[merged['Purchased']==0].head()
#filter where customer quanitity is 0
products_to_recommend = list(customer_orders['ProductName'].values)
products_to_recommend
#showing the values of the top 5 product recommendations for the customer id 33

['Wine - Redchard Merritt',
 'Bread - Calabrese Baguette',
 'Thyme - Lemon, Fresh',
 'Milk Powder',
 'Ecolab - Lime - A - Way 4/4 L']

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [14]:
all_recom = {}
unique_ids = list(data.CustomerID.unique())

for id in unique_ids:
    similar_customers = list(euclidean_sim[id].sort_values(ascending=False)[1:].head().index)
    similar_customer_records = df[df.CustomerID.isin(similar_customers)]
    grouped = pd.DataFrame(similar_customer_records.groupby('ProductName').Quantity.sum())
    product_records = grouped.sort_values('Quantity', ascending=False).reset_index()
    
    merged = pd.merge(product_records, pd.DataFrame(product_by_customer.T[id]), on='ProductName')
    merged.columns = ['ProductName', 'Quantity', 'Purchased']
    customer_orders = merged[merged['Purchased']==0].head()
    #filter where customer quanitity is 0
    products_to_recommend = list(customer_orders['ProductName'].values)

    all_recom[id] = ({'product1': products_to_recommend[0], 'product2': products_to_recommend[1],
                      'product3': products_to_recommend[2], 'product4': products_to_recommend[3],
                      'product5': products_to_recommend[4]})
print(all_recom)

{61288: {'product1': 'Ecolab - Solid Fusion', 'product2': 'Appetizer - Sausage Rolls', 'product3': 'Bread - English Muffin', 'product4': 'Tea - Jasmin Green', 'product5': 'Chocolate - Semi Sweet, Calets'}, 77352: {'product1': 'Soup - Campbells, Beef Barley', 'product2': 'Vaccum Bag 10x13', 'product3': 'Chicken - Soup Base', 'product4': 'Towels - Paper / Kraft', 'product5': 'Coffee Decaf Colombian'}, 40094: {'product1': 'Beef - Inside Round', 'product2': 'Vinegar - Sherry', 'product3': 'Onion Powder', 'product4': 'Beef - Top Sirloin', 'product5': 'Chocolate - Semi Sweet, Calets'}, 23548: {'product1': 'Soup - Campbells, Beef Barley', 'product2': 'Mussels - Cultivated', 'product3': 'Mushrooms - Black, Dried', 'product4': 'Bagel - Plain', 'product5': 'Salsify, Organic'}, 78981: {'product1': 'Scallops - 10/20', 'product2': 'Bouq All Italian - Primerba', 'product3': 'Olive - Spread Tapenade', 'product4': 'V8 - Berry Blend', 'product5': 'Chocolate - Dark'}, 83106: {'product1': 'Banana Turning

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [15]:
recommended_products_euc = pd.DataFrame(all_recom)
recommended_products_euc = recommended_products_euc.T.reset_index().rename(columns={'index': 'id'})
recommended_products_euc

,id,product1,product2,product3,product4,product5
0,61288,Ecolab - Solid Fusion,Appetizer - Sausage Rolls,Bread - English Muffin,Tea - Jasmin Green,"Chocolate - Semi Sweet, Calets"
1,77352,"Soup - Campbells, Beef Barley",Vaccum Bag 10x13,Chicken - Soup Base,Towels - Paper / Kraft,Coffee Decaf Colombian
2,40094,Beef - Inside Round,Vinegar - Sherry,Onion Powder,Beef - Top Sirloin,"Chocolate - Semi Sweet, Calets"
3,23548,"Soup - Campbells, Beef Barley",Mussels - Cultivated,"Mushrooms - Black, Dried",Bagel - Plain,"Salsify, Organic"
4,78981,Scallops - 10/20,Bouq All Italian - Primerba,Olive - Spread Tapenade,V8 - Berry Blend,Chocolate - Dark
...,...,...,...,...,...,...
995,49005,"Soup - Campbells, Beef Barley",Beans - Kidney White,Bread Crumbs - Japanese Style,Vaccum Bag 10x13,"Pepper - Black, Whole"
996,41286,Wine - Alsace Gewurztraminer,Cookie Dough - Double,Beef - Montreal Smoked Brisket,Knife Plastic - White,Coffee Decaf Colombian
997,85878,Scallops - 10/20,Pastry - Choclate Baked,Wiberg Super Cure,Table Cloth 54x72 White,"Bread - Roll, Canadian Dinner"
998,68506,Scallops - 10/20,Soupfoamcont12oz 112con,Barramundi,Eggplant - Asian,Olive - Spread Tapenade


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [16]:
def customer_sim(summary, metric='cosine'):
    pairwise = pd.DataFrame(
        (1 - squareform(pdist(summary, metric))),
        columns = summary.index,
        index = summary.index
    )
    return pairwise

In [17]:
#another way to calculate cosine similarity

#from sklearn.metrics.pairwise import cosine_similarity
#cosine_sim = cosine_similarity(product_by_customer, product_by_customer)

In [18]:
cosine_sim = customer_sim(product_by_customer)
cosine_sim

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.113219,0.225352,0.160228,0.156319,0.151757,0.084350,0.115991,0.124165,0.131865,...,0.114297,0.069451,0.104846,0.044856,0.127974,0.053751,0.108907,0.077693,0.111357,0.102980
200,0.113219,1.000000,0.138378,0.131185,0.081445,0.073933,0.204490,0.064750,0.135544,0.153111,...,0.165891,0.124063,0.198997,0.160257,0.114303,0.084015,0.255339,0.138786,0.099461,0.103476
264,0.225352,0.138378,1.000000,0.120171,0.182373,0.096572,0.144599,0.057995,0.137961,0.092305,...,0.171446,0.097232,0.131058,0.119616,0.191961,0.215002,0.176973,0.142437,0.197969,0.102980
356,0.160228,0.131185,0.120171,1.000000,0.098796,0.091552,0.137082,0.151197,0.065394,0.075006,...,0.135445,0.079009,0.124245,0.127573,0.109189,0.114652,0.077434,0.135033,0.164218,0.134236
412,0.156319,0.081445,0.182373,0.098796,1.000000,0.127598,0.156028,0.174328,0.114839,0.158548,...,0.105713,0.231245,0.109093,0.110632,0.153870,0.136712,0.075545,0.143715,0.171656,0.190490
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.053751,0.084015,0.215002,0.114652,0.136712,0.157949,0.149455,0.124497,0.131625,0.100647,...,0.122679,0.145775,0.087527,0.156919,0.244193,1.000000,0.220798,0.160604,0.118048,0.073688
98069,0.108907,0.255339,0.176973,0.077434,0.075545,0.080007,0.128115,0.098097,0.120011,0.152944,...,0.110474,0.134255,0.114006,0.072259,0.197908,0.220798,1.000000,0.175219,0.155469,0.136860
98159,0.077693,0.138786,0.142437,0.135033,0.143715,0.126837,0.132940,0.133298,0.164888,0.133355,...,0.157622,0.204324,0.132540,0.082479,0.152952,0.160604,0.175219,1.000000,0.159256,0.118345


In [19]:
all_recom_cos = {}
unique_ids = list(data.CustomerID.unique())

for id in unique_ids:
    similar_customers = list(cosine_sim[id].sort_values(ascending=False)[1:].head().index)
    similar_customer_records = df[df.CustomerID.isin(similar_customers)]
    grouped = pd.DataFrame(similar_customer_records.groupby('ProductName').Quantity.sum())
    product_records = grouped.sort_values('Quantity', ascending=False).reset_index()
    
    merged = pd.merge(product_records, pd.DataFrame(product_by_customer.T[id]), on='ProductName')
    merged.columns = ['ProductName', 'Quantity', 'Purchased']
    customer_orders = merged[merged['Purchased']==0].head()
    #filter where customer quanitity is 0
    products_to_recommend = list(customer_orders['ProductName'].values)

    all_recom_cos[id] = ({'product1': products_to_recommend[0], 'product2': products_to_recommend[1],
                      'product3': products_to_recommend[2], 'product4': products_to_recommend[3],
                      'product5': products_to_recommend[4]})
print(all_recom_cos)

{61288: {'product1': 'Jagermeister', 'product2': 'Wine - Two Oceans Cabernet', 'product3': 'Tofu - Firm', 'product4': 'Appetizer - Sausage Rolls', 'product5': 'Dc - Frozen Momji'}, 77352: {'product1': 'Bread - Rye', 'product2': 'Fondant - Icing', 'product3': 'Lettuce - Frisee', 'product4': 'Bread - French Baquette', 'product5': 'Pork - Kidney'}, 40094: {'product1': 'Juice - V8 Splash', 'product2': 'Beef - Inside Round', 'product3': 'Bread - Roll, Soft White Round', 'product4': 'Truffle Cups - Brown', 'product5': 'Cake - Mini Cheesecake'}, 23548: {'product1': 'Juice - Orange', 'product2': 'Bagel - Plain', 'product3': 'Olives - Stuffed', 'product4': 'Veal - Sweetbread', 'product5': 'Cinnamon Buns Sticky'}, 78981: {'product1': 'Wine - Blue Nun Qualitatswein', 'product2': 'Cookies - Assorted', 'product3': 'Cheese - Cottage Cheese', 'product4': 'Baking Powder', 'product5': 'Quiche Assorted'}, 83106: {'product1': 'Bouq All Italian - Primerba', 'product2': 'Onions - Vidalia', 'product3': 'Sca

In [20]:
recommended_products_cos = pd.DataFrame(all_recom_cos)
recommended_products_cos = recommended_products_cos.T.reset_index().rename(columns={'index': 'id'})
recommended_products_cos

,id,product1,product2,product3,product4,product5
0,61288,Jagermeister,Wine - Two Oceans Cabernet,Tofu - Firm,Appetizer - Sausage Rolls,Dc - Frozen Momji
1,77352,Bread - Rye,Fondant - Icing,Lettuce - Frisee,Bread - French Baquette,Pork - Kidney
2,40094,Juice - V8 Splash,Beef - Inside Round,"Bread - Roll, Soft White Round",Truffle Cups - Brown,Cake - Mini Cheesecake
3,23548,Juice - Orange,Bagel - Plain,Olives - Stuffed,Veal - Sweetbread,Cinnamon Buns Sticky
4,78981,Wine - Blue Nun Qualitatswein,Cookies - Assorted,Cheese - Cottage Cheese,Baking Powder,Quiche Assorted
...,...,...,...,...,...,...
995,49005,Squid - Tubes / Tenticles 10/20,"Pepper - White, Ground",Soup Knorr Chili With Beans,Cheese - Bocconcini,Barramundi
996,41286,"Hickory Smoke, Liquid","Salsify, Organic",Dc Hikiage Hira Huba,Oil - Shortening - All - Purpose,Banana - Leaves
997,85878,Bread - Calabrese Baguette,French Pastry - Mini Chocolate,Cheese - Parmesan Grated,Foam Dinner Plate,Cheese - Mix
998,68506,Flavouring - Orange,Longos - Chicken Wings,"Cheese - Brie, Triple Creme",Tia Maria,Fondant - Icing
